In [1]:
! pip install Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


In [7]:
train_df = pd.read_csv('D:/KSM Android - DS Beginner/Pertemuan9/fake-news-classification/train (2).csv', on_bad_lines='skip',sep=';')
train_df.head()

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0


In [8]:
train_df = train_df[['title', 'text', 'label']]
train_df['content'] = train_df['title'] + " " + train_df['text']

train_df.head()

,title,text,label,content
0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1,Palestinians switch off Christmas lights in Be...
1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1,China says Trump call with Taiwan president wo...
2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0,FAIL! The Trump Organization’s Credit Score W...
3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1,Zimbabwe military chief's China trip was norma...
4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = train_df['content']
y = train_df['label']

tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
review_tf = tfidf_vectorizer.fit_transform(X.astype('U'))
review_tf

<24353x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3224666 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(review_tf, y, train_size = 0.8, test_size = 0.2, random_state = 42)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf_rf = RandomForestClassifier().fit(X_train, y_train)
predicted_rf = clf_rf.predict(X_test)

rf_accuracy = accuracy_score(y_test, predicted_rf)

print("Random Forest Accuracy: ", accuracy_score(y_test, predicted_rf))
print("Random Forest Precision: ", precision_score(y_test, predicted_rf, average = "macro"))
print("Random Forest Recall: ", recall_score(y_test, predicted_rf, average = "macro"))
print("Random Forest F1 Score: ", f1_score(y_test, predicted_rf, average = "macro"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted_rf)}')
print("----------------------------------------------------------")
print(classification_report(y_test, predicted_rf, zero_division = 0))

Random Forest Accuracy:  0.9790597413262164
Random Forest Precision:  0.9781121197605196
Random Forest Recall:  0.9801235415703446
Random Forest F1 Score:  0.9789348909116318
confusion matrix:
 [[2197   18]
 [  84 2572]]
----------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      2215
           1       0.99      0.97      0.98      2656

    accuracy                           0.98      4871
   macro avg       0.98      0.98      0.98      4871
weighted avg       0.98      0.98      0.98      4871



In [12]:
valid_df = pd.read_csv('D:/KSM Android - DS Beginner/Pertemuan9/fake-news-classification/evaluation.csv', on_bad_lines='skip',sep=';')

valid_df = valid_df[['title', 'text', 'label']]
valid_df['content'] = valid_df['title'] + " " + valid_df['text']

validation_X = valid_df['content']
validation_tf = tfidf_vectorizer.transform(validation_X.astype('U'))

validation_predictions = clf_rf.predict(validation_tf)

if 'label' in valid_df.columns:
    validation_y = valid_df['label']
    print("Validation Accuracy: ", accuracy_score(validation_y, validation_predictions))
    print("Validation Precision: ", precision_score(validation_y, validation_predictions, average='macro'))
    print("Validation Recall: ", recall_score(validation_y, validation_predictions, average='macro'))
    print("Validation F1 Score: ", f1_score(validation_y, validation_predictions, average='macro'))
    print(f'Confusion matrix:/n{confusion_matrix(validation_y, validation_predictions)}')
    print(classification_report(validation_y, validation_predictions, zero_division=0))

valid_df['Predicted Label'] = validation_predictions
valid_df.head()

Validation Accuracy:  0.9793026980411482
Validation Precision:  0.9787948666350123
Validation Recall:  0.9800301618747418
Validation F1 Score:  0.9792505299753316
Confusion matrix:/n[[3771   32]
 [ 136 4178]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3803
           1       0.99      0.97      0.98      4314

    accuracy                           0.98      8117
   macro avg       0.98      0.98      0.98      8117
weighted avg       0.98      0.98      0.98      8117



,title,text,label,content,Predicted Label
0,"Sanders back in U.S. Senate, blasts 'coloniali...",WASHINGTON (Reuters) - Democratic U.S. preside...,1,"Sanders back in U.S. Senate, blasts 'coloniali...",1
1,Kremlin: Syria peoples' congress being 'active...,MOSCOW (Reuters) - A proposal to convene a con...,1,Kremlin: Syria peoples' congress being 'active...,1
2,Oregon Cop Convicted Of Shattering Biker’s Co...,"In a baffling fit of rage, an Oregon State Pol...",0,Oregon Cop Convicted Of Shattering Biker’s Co...,0
3,Twitter Erupts With Glee Over #CruzSexScandal...,The last thing any politician running for the ...,0,Twitter Erupts With Glee Over #CruzSexScandal...,0
4,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,This is too good to miss! Mr. Teleprompter did...,0,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,0


In [13]:
import pickle

# Simpan TF-IDF Vectorizer
with open('tfidf_vectorizer.sav', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print("TF-IDF Vectorizer saved to 'tfidf_vectorizer.sav'")

# Simpan Random Forest Model
with open('random_forest_model.sav', 'wb') as f:
    pickle.dump(clf_rf, f)
print("Random Forest model saved to 'random_forest_model.sav'")

TF-IDF Vectorizer saved to 'tfidf_vectorizer.sav'
Random Forest model saved to 'random_forest_model.sav'


# Testing

In [ ]:
# Load TF-IDF Vectorizer
with open('tfidf_vectorizer.sav', 'rb') as f:
    tfidf_vectorizer_loaded = pickle.load(f)
print("TF-IDF Vectorizer loaded successfully.")

# Load Random Forest Model
with open('random_forest_model.sav', 'rb') as f:
    clf_rf_loaded = pickle.load(f)
print("Random Forest model loaded successfully.")

In [ ]:
test_df = pd.read_csv('D:/KSM Android - DS Beginner/Pertemuan9/fake-news-classification/test (1).csv', on_bad_lines='skip',sep=';')

test_df = test_df[['title', 'text', 'label']]
test_df['content'] = test_df['title'] + " " + test_df['text']

test_X = test_df['content']
test_tf = tfidf_vectorizer_loaded.transform(test_X.astype('U'))

test_predictions = clf_rf_loaded.predict(test_tf)

if 'label' in test_df.columns:
    test_y = test_df['label']
    print("Test Accuracy: ", accuracy_score(test_y, test_predictions))
    print("Test Precision: ", precision_score(test_y, test_predictions, average='macro'))
    print("Test Recall: ", recall_score(test_y, test_predictions, average='macro'))
    print("Test F1 Score: ", f1_score(test_y, test_predictions, average='macro'))
    print(f'Confusion matrix:/n{confusion_matrix(test_y, test_predictions)}')
    print(classification_report(test_y, test_predictions, zero_division=0))

test_df['Predicted Label'] = test_predictions
test_df.head()

Test Accuracy:  0.9823826536897868
Test Precision:  0.981790841337677
Test Recall:  0.983000400779285
Test F1 Score:  0.9823060736412004
Confusion matrix:
[[3720   33]
 [ 110 4254]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      3753
           1       0.99      0.97      0.98      4364

    accuracy                           0.98      8117
   macro avg       0.98      0.98      0.98      8117
weighted avg       0.98      0.98      0.98      8117



,title,text,label,content,Predicted Label
0,"Live from New York, it's a Trump-Clinton remat...",NEW YORK (Reuters) - Veteran actor and frequen...,1,"Live from New York, it's a Trump-Clinton remat...",1
1,Catalan separatists to lose majority in tight ...,BARCELONA (Reuters) - Catalonia s independence...,1,Catalan separatists to lose majority in tight ...,1
2,North Carolina governor concedes election to D...,"WINSTON-SALEM, N.C. (Reuters) - North Carolina...",1,North Carolina governor concedes election to D...,1
3,Draft Senate Iran legislation sets tough new U...,WASHINGTON (Reuters) - Draft legislation respo...,1,Draft Senate Iran legislation sets tough new U...,1
4,California governor taps U.S. Representative B...,"SACRAMENTO, Calif. (Reuters) - California Gove...",1,California governor taps U.S. Representative B...,1


In [1]:
!pip freeze > requirements.txt